# <center> Полезные функции

<center> Сюда буду добавлять полезные функции, которые могут пригодиться в будущем. В комментарии может быть код примера работы или дополнительные фичи.

In [ ]:
from sklearn.cluster import KMeans

# функция, которая принимает количество кластеров для k-means и матрицу с признаками объектов и возвращает инерцию 
def get_inertia(cluster_num, X):
# инициализируем алгоритм кластеризации
    k_means =  KMeans(n_clusters=cluster_num, random_state=42)
# запускаем алгоритм k-means
    k_means.fit(X)
# находим значение инерции
    inertia = k_means.inertia_
# возвращаем значение инерции
    return inertia

# # создаём пустой список для значений инерции
# inertia = []
# # итерируемся по разным размерам кластеров (от 1 до 9) и сохраняем значение инерции для каждого кластера
# for cluster_num in range(1, 10):
# # сохраняем значения
#     inertia.append(get_inertia(cluster_num, X))

# # визуализируем, как менялась инерция в зависимости от количества кластеров
# # задаём названия осям x и y
# plt.xlabel("cluster", fontsize=12)
# plt.ylabel("inertia", fontsize=12)
# # рисуем изменение инерции
# plt.plot([i for i in range(1, 10)], inertia, 'xb-')

In [ ]:
# импортируем метрику силуэта
from sklearn.metrics import silhouette_score

# напишем функцию, как и при подсчете метода локтя
def get_silhouette(cluster_num, X):
    k_means = KMeans(n_clusters=cluster_num, init='k-means++', n_init=10, random_state=42)
    k_means.fit(X)
# подсчитаем метрику силуэта, передав данные и то, к каким кластерам относятся объекты
    silhouette = silhouette_score(X, k_means.predict(X))
    return silhouette

# # создадим пустой словарь, ключами будут инерция и количество кластеров
# silhouette_res = {"silhouette": [], "cluster": []}

# # выберем нужные данные 
# X = df[['Attack', 'Defense']]

# for cluster_num in range(2, 10):
#     silhouette_res["silhouette"].append(get_silhouette(cluster_num, X))
#     silhouette_res["cluster"].append(cluster_num)
    
# # сохраним в датафрейм значение силуэта и количество кластеров
# silhouette_df = pd.DataFrame(silhouette_res)

# # установим стиль для визуализиции
# sns.set_style("darkgrid")
# # визуализируем зависимость значения инерции от количества кластеров
# sns.lineplot(data=silhouette_df, x="cluster", y="silhouette", marker= "o")

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import seaborn as sns

# напишем функцию, которая будет проводить алгомеративную кластеризацию
# на вход она будет принимать X — матрицу с признаками для кластеризации и n_clusters — количество кластеров,
# на выходе будет возвращать список с номерами кластеров
def get_aggl_clustering(X, n_clusters):
    # запустим агломеративную кластеризацию
    agglomerative_clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
    # обучим модель агломеративной кластеризации
    agglomerative_clustering.fit(X)
    # получим список c информацией, к какому кластеру относятся объекты
    aggl_prediction = agglomerative_clustering.labels_
    # вернём список с результатами
    return aggl_prediction

# напишем функцию для подсчёта коэффициента силуэта
def get_silhouette_agg_clust(X, cluster_num):
    # проведём агломеративную кластеризацию с заданным количеством кластеров
    res = get_aggl_clustering(X, cluster_num)
    # рассчитаем коэффициент силуэта
    silhouette = silhouette_score(X, res)
    return silhouette

In [ ]:
#Метод межквартильного размаха (метод Тьюки)
#Модифицированная функция с настраиваемыми IQR, и проверкой на логарифмирование
#Подробнее в PY-14
import numpy as np

def outliers_iqr_mod(data, feature, log_scale=False, left=1.5, right=1.5):
    
    """Функция для очистки дата-фрейма от выбросов по методу межквартильного размаха (метода Тьюки).

    Args:
        data (DataFrame): дата-фрейм с данными.
        feature (pd.Series): признак (столбец) из дата-фрейма. 
        log_scale (bool, optional): Логарифмирование в процессе работы функции. Defaults to False.
        left (float, optional): левый интервал, за пределами которого значения считаются выбросами. Defaults to 1.5.
        right (float, optional): правый интервал, за пределами которого значения считаются выбросами. Defaults to 1.5.

    Returns:
        outliners (DataFrame): дата-фрейм со строками, которые функция посчитала выбросами.
        cleaned (DataFtame): очищенный от выбросов дата-фрейм. 
    """    
    
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75)
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliners = data[(x < lower_bound)|(x > upper_bound)]
    cleaned = data[(x >= lower_bound)&(x <= upper_bound)]
    return outliners, cleaned

In [ ]:
#метод z-отклонений (метод сигм)
import numpy as np

def outliers_z_score_mod(data, feature, log_scale=False,
                         left=3, right=3):
    
    """Функция для очистки дата-фрейма от выбросов по методу z-отклонений (методу сигм)

    Args:
        data (DataFrame): дата-фрейм с данными.
        feature (pd.Series): признак (столбец) из дата-фрейма. 
        log_scale (bool, optional): Логарифмирование в процессе работы функции. Defaults to False.
        left (float, optional): левый интервал, за пределами которого значения считаются выбросами. Defaults to 3.
        right (float, optional): правый интервал, за пределами которого значения считаются выбросами. Defaults to 3.

    Returns:
        outliners (DataFrame): дата-фрейм со строками, которые функция посчитала выбросами.
        cleaned (DataFtame): очищенный от выбросов дата-фрейм. 
    """    
    
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left*sigma
    upper_bound = mu + right*sigma
    outliers = data[(x<lower_bound) | (x>upper_bound)]
    cleaned = data[(x>lower_bound) & (x<upper_bound)]
    return outliers, cleaned

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def show_corr_heatmap(data, columns_list, title, fontsize=18, method='pearson'):
    
    """Функция для построения тепловой карты корреляций столбцов дата-фрейма.

    Args:
        data (DataFrame): дата-фрейм, по которому нужно построить тепловую карту.
        columns_list (list): список столбцов, по которым нужно построить тепловую карту.
        title (str): будущий заголовок тепловой карты.
        fontsize (int): размер шрифта для заголовка. По умолчанию равен 18.
        method (str, optional): метод корреляции признаков. По умолчанию используется корреляция Пирсона.
    """    
    
    fig_, ax_ = plt.subplots(figsize=(15, 15))
    corr = data[columns_list].corr(method)
    mask = np.triu(np.ones_like(corr, dtype=bool))
    sns.heatmap(corr,
                annot=True,
                linewidths=0.1,
                ax=ax_,
                mask=mask,
                cmap='viridis',
                fmt='.1g')
    ax_.set_title(title, fontsize=fontsize)
    plt.show()

In [ ]:
from sklearn import model_selection
import numpy as np

def plot_learning_curve(model, X, y, cv, scoring="f1", ax=None, title=""):
    
    """Функция для построения графика кривых обучения.

    Args:
        model (sklearn): Модель, для которой строится график.
        X (pd.DataFrame): Набор параметров на вход для модели.
        y (pd.Series): Значения целевого признака обучения.
        cv (sklearn.model_selection): Кросс-валидатор
        scoring (str, optional): Метрика оценивания модели. Defaults to "f1".
        ax (_type_, optional): Координатная плоскость matplotlib. Defaults to None.
        title (str, optional): Заголовок графика. Defaults to "".
    """    
    
    # Вычисляем координаты для построения кривой обучения
    train_sizes, train_scores, valid_scores = model_selection.learning_curve(
        estimator=model,  # модель
        X=X,  # матрица наблюдений X
        y=y,  # вектор ответов y
        cv=cv,  # кросс-валидатор
        scoring=scoring,  # метрика
    )
    # Вычисляем среднее значение по фолдам для каждого набора данных
    train_scores_mean = np.mean(train_scores, axis=1)
    valid_scores_mean = np.mean(valid_scores, axis=1)
    # Если координатной плоскости не было передано, создаём новую
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 4))  # фигура + координатная плоскость
    # Строим кривую обучения по метрикам на тренировочных фолдах
    ax.plot(train_sizes, train_scores_mean, label="Train")
    # Строим кривую обучения по метрикам на валидационных фолдах
    ax.plot(train_sizes, valid_scores_mean, label="Valid")
    # Даём название графику и подписи осям
    ax.set_title("Learning curve: {}".format(title))
    ax.set_xlabel("Train data size")
    ax.set_ylabel("Score")
    # Устанавливаем отметки по оси абсцисс
    ax.xaxis.set_ticks(train_sizes)
    # Устанавливаем диапазон оси ординат
    ax.set_ylim(0, 1)
    # Отображаем легенду
    ax.legend()

In [ ]:
#Для оптимизации с hyperopt
def hyperopt_rf(params, cv=5, X=X_train_scaled, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    # score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score